# 03 - Modelagem e Treinamento

Neste notebook vamos desenvolver e treinar modelos de machine learning para previsão de vendas semanais.

## Objetivos:
1. **Carregamento dos Dados**: Carregar dataset com features processadas
2. **Preparação para ML**: Dividir dados em treino/validação, preparar features
3. **Baseline Models**: Implementar modelos simples como referência
4. **Advanced Models**: Treinar modelos avançados (LightGBM, XGBoost, etc.)
5. **Ensemble**: Combinar múltiplos modelos para melhor performance
6. **Validação**: Avaliar performance usando métricas adequadas
7. **Predições Finais**: Gerar previsões para período de teste

In [ ]:
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Advanced ML
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Time series
from datetime import datetime, timedelta
import gc

plt.style.use('default')
sns.set_palette("husl")

print('📚 Bibliotecas carregadas com sucesso!')
print('🎯 Iniciando fase de Modelagem e Treinamento')

## 1. Carregamento dos Dados Processados

In [ ]:
# Carregar dados com features processadas
print('📂 Carregando dados processados...')

# Verificar se os arquivos essenciais existem
import os
required_files = [
    '../data/dados_features_completo.parquet',  # Usar parquet (mais rápido)
    '../data/feature_engineering_metadata.pkl'
]

missing_files = [f for f in required_files if not os.path.exists(f)]
if missing_files:
    print('❌ Arquivos não encontrados:')
    for f in missing_files:
        print(f'   • {f}')
    print('\n🔄 Execute primeiro o notebook 02-Feature-Engineering-Dask.ipynb')
else:
    print('✅ Todos os arquivos necessários encontrados')
    
    # Carregar dados principais (usar parquet para velocidade)
    print('📊 Carregando dataset (parquet é 57x mais rápido que CSV)...')
    dados = pd.read_parquet('../data/dados_features_completo.parquet')
    
    # Carregar metadados
    with open('../data/feature_engineering_metadata.pkl', 'rb') as f:
        metadata = pickle.load(f)
    
    print(f'\n📊 Dados carregados com sucesso:')
    print(f'   • Shape: {dados.shape}')
    print(f'   • Período: {dados["semana"].min()} até {dados["semana"].max()}')
    print(f'   • Features disponíveis: {len(dados.columns)}')
    print(f'   • Memória: {dados.memory_usage(deep=True).sum() / (1024**2):.1f} MB')
    print(f'   • Estratégia: {metadata.get("estrategia", "Grid Inteligente")}')
    
    print(f'\n🔍 Metadados do processamento:')
    for key, value in metadata.items():
        if key not in ['features_criadas', 'data_processamento']:  # Skip long items
            print(f'   • {key}: {value}')
    
    print(f'\n✅ Pronto para modelagem!')

## 2. Preparação dos Dados para ML

In [ ]:
# Definir variável target e features
target = 'quantidade'

# Features a excluir (não devem ser usadas para predição)
exclude_features = [
    'pdv_id', 'produto_id', 'semana',  # IDs e data
    'quantidade',  # Target
    'valor', 'num_transacoes',  # Features que vazam informação do futuro
    'semana_primeira_venda', 'semana_ultima_venda',  # Dates
]

# Identificar features disponíveis
all_features = [col for col in dados.columns if col not in exclude_features]

print(f'🎯 Preparação dos dados:')
print(f'   • Target: {target}')
print(f'   • Features disponíveis: {len(all_features)}')
print(f'   • Features excluídas: {len(exclude_features)}')

# Verificar missing values nas features
missing_features = dados[all_features].isnull().sum()
missing_features = missing_features[missing_features > 0].sort_values(ascending=False)

if len(missing_features) > 0:
    print(f'\n⚠️ Features com valores missing:')
    for feature, count in missing_features.head(10).items():
        pct = (count / len(dados)) * 100
        print(f'   • {feature}: {count:,} ({pct:.1f}%)')
else:
    print('\n✅ Nenhum valor missing nas features')

# Remover features com muitos missing values (>50%)
high_missing = missing_features[missing_features > len(dados) * 0.5].index.tolist()
if high_missing:
    print(f'\n🗑️ Removendo features com >50% missing: {len(high_missing)}')
    all_features = [f for f in all_features if f not in high_missing]

print(f'\n📋 Features finais para modelagem: {len(all_features)}')

In [ ]:
# Divisão temporal dos dados (Time Series Split)
# Usar últimas 8 semanas para validação
print('📅 Divisão temporal dos dados...')

# Ordenar por semana
dados_sorted = dados.sort_values('semana')

# Identificar ponto de corte
semanas_unicas = sorted(dados_sorted['semana'].unique())
n_semanas_val = 8  # Últimas 8 semanas para validação
cutoff_week = semanas_unicas[-n_semanas_val]

# Dividir dados
train_data = dados_sorted[dados_sorted['semana'] < cutoff_week].copy()
val_data = dados_sorted[dados_sorted['semana'] >= cutoff_week].copy()

print(f'📊 Divisão dos dados:')
print(f'   • Treino: {len(train_data):,} registros ({len(train_data)/len(dados)*100:.1f}%)')
print(f'   • Validação: {len(val_data):,} registros ({len(val_data)/len(dados)*100:.1f}%)')
print(f'   • Semanas treino: {train_data["semana"].nunique()}')
print(f'   • Semanas validação: {val_data["semana"].nunique()}')
print(f'   • Cutoff: {cutoff_week}')

# Preparar features e targets
X_train = train_data[all_features].fillna(0)
y_train = train_data[target]
X_val = val_data[all_features].fillna(0)
y_val = val_data[target]

print(f'\n✅ Dados preparados para modelagem')
print(f'   • X_train shape: {X_train.shape}')
print(f'   • X_val shape: {X_val.shape}')

## 3. Análise Exploratória do Target

In [ ]:
# Análise da distribuição do target
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Distribuição geral
axes[0,0].hist(y_train, bins=50, alpha=0.7, edgecolor='black')
axes[0,0].set_title('Distribuição da Quantidade (Treino)')
axes[0,0].set_xlabel('Quantidade')
axes[0,0].set_ylabel('Frequência')

# Log-scale
non_zero_train = y_train[y_train > 0]
axes[0,1].hist(np.log1p(non_zero_train), bins=50, alpha=0.7, edgecolor='black')
axes[0,1].set_title('Distribuição log(Quantidade + 1) - Apenas > 0')
axes[0,1].set_xlabel('log(Quantidade + 1)')
axes[0,1].set_ylabel('Frequência')

# Zeros vs Non-zeros
zero_counts = [len(y_train[y_train == 0]), len(y_train[y_train > 0])]
axes[1,0].pie(zero_counts, labels=['Zeros', 'Não-zeros'], autopct='%1.1f%%')
axes[1,0].set_title('Proporção Zeros vs Não-zeros')

# Boxplot por semana (últimas 12 semanas)
recent_weeks = train_data['semana'].nlargest(12*len(train_data)).unique()
recent_data = train_data[train_data['semana'].isin(recent_weeks)]
recent_data.boxplot('quantidade', by='semana', ax=axes[1,1])
axes[1,1].set_title('Distribuição por Semana (Últimas 12)')
axes[1,1].set_xlabel('Semana')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Estatísticas descritivas
print('📈 Estatísticas do Target (treino):')
print(y_train.describe())

print(f'\n🎯 Métricas importantes:')
print(f'   • Zeros: {(y_train == 0).sum():,} ({(y_train == 0).mean()*100:.1f}%)')
print(f'   • Não-zeros: {(y_train > 0).sum():,} ({(y_train > 0).mean()*100:.1f}%)')
print(f'   • Média (apenas > 0): {y_train[y_train > 0].mean():.2f}')
print(f'   • Mediana (apenas > 0): {y_train[y_train > 0].median():.2f}')

## 4. Modelos Baseline

In [ ]:
# Função para avaliar modelos
def evaluate_model(y_true, y_pred, model_name):
    """
    Avalia um modelo usando múltiplas métricas incluindo WMAPE
    """
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    # WMAPE - Weighted Mean Absolute Percentage Error (métrica oficial do challenge)
    wmape = np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true)) * 100
    
    # Métricas para zeros vs não-zeros
    zero_mask = y_true == 0
    nonzero_mask = y_true > 0
    
    mae_zero = mean_absolute_error(y_true[zero_mask], y_pred[zero_mask]) if zero_mask.sum() > 0 else 0
    mae_nonzero = mean_absolute_error(y_true[nonzero_mask], y_pred[nonzero_mask]) if nonzero_mask.sum() > 0 else 0
    
    results = {
        'Model': model_name,
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2,
        'WMAPE': wmape,
        'MAE_Zero': mae_zero,
        'MAE_NonZero': mae_nonzero
    }
    
    return results

# Lista para armazenar resultados
model_results = []

print('🎯 Treinando Modelos Baseline...')

# 1. Baseline: Média Simples
print('\n📊 1. Baseline - Média Simples')
mean_pred = np.full(len(y_val), y_train.mean())
results_mean = evaluate_model(y_val, mean_pred, 'Média Simples')
model_results.append(results_mean)

# 2. Baseline: Média por combinação PDV/produto
print('📊 2. Baseline - Média por Combinação')
combo_means = train_data.groupby(['pdv_id', 'produto_id'])['quantidade'].mean().to_dict()
global_mean = y_train.mean()

combo_pred = []
for _, row in val_data.iterrows():
    key = (row['pdv_id'], row['produto_id'])
    pred = combo_means.get(key, global_mean)
    combo_pred.append(pred)

combo_pred = np.array(combo_pred)
results_combo = evaluate_model(y_val, combo_pred, 'Média por Combinação')
model_results.append(results_combo)

# 3. Baseline: Last Value (usar última quantidade conhecida)
print('📊 3. Baseline - Último Valor')
last_values = train_data.groupby(['pdv_id', 'produto_id'])['quantidade'].last().to_dict()

last_pred = []
for _, row in val_data.iterrows():
    key = (row['pdv_id'], row['produto_id'])
    pred = last_values.get(key, global_mean)
    last_pred.append(pred)

last_pred = np.array(last_pred)
results_last = evaluate_model(y_val, last_pred, 'Último Valor')
model_results.append(results_last)

# Mostrar resultados dos baselines
baseline_df = pd.DataFrame(model_results)
print('\n📋 Resultados dos Baselines:')
print(baseline_df.round(4))

print(f'\n🎯 WMAPE dos Baselines:')
for result in model_results:
    print(f'   • {result["Model"]}: {result["WMAPE"]:.2f}%')

print('\n✅ Baselines estabelecidos - qualquer modelo ML deve superar estes resultados!')

## 5. Modelos de Machine Learning

In [ ]:
print('🤖 Treinando Modelos de Machine Learning...')

# 4. Random Forest
print('\n🌲 4. Random Forest')
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_val)
rf_pred = np.maximum(0, rf_pred)  # Não permitir previsões negativas

results_rf = evaluate_model(y_val, rf_pred, 'Random Forest')
model_results.append(results_rf)

# Feature importance (top 10)
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print('🔝 Top 10 features mais importantes (Random Forest):')
for i, (_, row) in enumerate(feature_importance.head(10).iterrows(), 1):
    print(f'   {i:2d}. {row["feature"]}: {row["importance"]:.4f}')


In [ ]:
# 5. LightGBM
print('\n💡 5. LightGBM')

# Preparar dados para LightGBM
train_lgb = lgb.Dataset(X_train, label=y_train)
val_lgb = lgb.Dataset(X_val, label=y_val, reference=train_lgb)

# Parâmetros LightGBM
lgb_params = {
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 20,
    'lambda_l1': 0.1,
    'lambda_l2': 0.1,
    'random_state': 42,
    'verbosity': -1
}

# Treinar modelo
lgb_model = lgb.train(
    lgb_params,
    train_lgb,
    num_boost_round=1000,
    valid_sets=[train_lgb, val_lgb],
    valid_names=['train', 'eval'],
    early_stopping_rounds=50,
    verbose_eval=False
)

# Predições
lgb_pred = lgb_model.predict(X_val, num_iteration=lgb_model.best_iteration)
lgb_pred = np.maximum(0, lgb_pred)

results_lgb = evaluate_model(y_val, lgb_pred, 'LightGBM')
model_results.append(results_lgb)

print(f'✅ LightGBM treinado - Melhor iteração: {lgb_model.best_iteration}')

In [ ]:
# 6. XGBoost
print('\n🚀 6. XGBoost')

# Parâmetros XGBoost
xgb_params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'mae',
    'max_depth': 6,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 10,
    'alpha': 0.1,
    'lambda': 0.1,
    'random_state': 42,
    'verbosity': 0
}

# Treinar modelo
xgb_model = xgb.XGBRegressor(**xgb_params, n_estimators=1000)
xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    early_stopping_rounds=50,
    verbose=False
)

# Predições
xgb_pred = xgb_model.predict(X_val)
xgb_pred = np.maximum(0, xgb_pred)

results_xgb = evaluate_model(y_val, xgb_pred, 'XGBoost')
model_results.append(results_xgb)

print(f'✅ XGBoost treinado - Melhor iteração: {xgb_model.best_iteration}')

## 6. Comparação de Modelos

In [ ]:
# Comparar todos os modelos
results_df = pd.DataFrame(model_results)
results_df = results_df.sort_values('MAE')

print('🏆 Ranking de Modelos por MAE:')
print('=' * 80)
print(results_df.round(4).to_string(index=False))

# Visualização dos resultados
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# MAE comparison
results_df.plot(x='Model', y='MAE', kind='bar', ax=axes[0], color='skyblue')
axes[0].set_title('Mean Absolute Error por Modelo')
axes[0].set_ylabel('MAE')
axes[0].tick_params(axis='x', rotation=45)

# R² comparison
results_df.plot(x='Model', y='R²', kind='bar', ax=axes[1], color='lightcoral')
axes[1].set_title('R² por Modelo')
axes[1].set_ylabel('R²')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Selecionar melhor modelo
best_model_name = results_df.iloc[0]['Model']
print(f'\n🥇 Melhor modelo: {best_model_name}')
print(f'   • MAE: {results_df.iloc[0]["MAE"]:.4f}')
print(f'   • RMSE: {results_df.iloc[0]["RMSE"]:.4f}')
print(f'   • R²: {results_df.iloc[0]["R²"]:.4f}')

## 7. Análise de Erros do Melhor Modelo

In [ ]:
# Usar as previsões do melhor modelo para análise
if best_model_name == 'LightGBM':
    best_pred = lgb_pred
    best_model = lgb_model
elif best_model_name == 'XGBoost':
    best_pred = xgb_pred
    best_model = xgb_model
elif best_model_name == 'Random Forest':
    best_pred = rf_pred
    best_model = rf_model
else:
    # Fallback para baseline
    best_pred = combo_pred
    best_model = None

# Análise de erros
errors = y_val - best_pred
abs_errors = np.abs(errors)

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Distribuição dos erros
axes[0,0].hist(errors, bins=50, alpha=0.7, edgecolor='black')
axes[0,0].set_title('Distribuição dos Erros')
axes[0,0].set_xlabel('Erro (Real - Predito)')
axes[0,0].set_ylabel('Frequência')
axes[0,0].axvline(0, color='red', linestyle='--', alpha=0.7)

# Scatter: Real vs Predito
sample_idx = np.random.choice(len(y_val), min(5000, len(y_val)), replace=False)
axes[0,1].scatter(y_val.iloc[sample_idx], best_pred[sample_idx], alpha=0.5)
axes[0,1].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--')
axes[0,1].set_title('Real vs Predito (amostra)')
axes[0,1].set_xlabel('Valor Real')
axes[0,1].set_ylabel('Valor Predito')

# Erros por faixa de valor real
val_bins = pd.cut(y_val, bins=10, labels=False)
error_by_bin = [abs_errors[val_bins == i].mean() for i in range(10)]
axes[1,0].bar(range(10), error_by_bin)
axes[1,0].set_title('MAE por Faixa de Valor Real')
axes[1,0].set_xlabel('Faixa (0=menor, 9=maior)')
axes[1,0].set_ylabel('MAE')

# Residuals plot
axes[1,1].scatter(best_pred[sample_idx], errors.iloc[sample_idx], alpha=0.5)
axes[1,1].axhline(0, color='red', linestyle='--', alpha=0.7)
axes[1,1].set_title('Residuais vs Predições')
axes[1,1].set_xlabel('Valor Predito')
axes[1,1].set_ylabel('Erro')

plt.tight_layout()
plt.show()

# Estatísticas dos erros
print(f'📊 Análise de Erros - {best_model_name}:')
print(f'   • Erro médio: {errors.mean():.4f}')
print(f'   • Erro absoluto médio: {abs_errors.mean():.4f}')
print(f'   • Desvio padrão dos erros: {errors.std():.4f}')
print(f'   • % predições exatas (zeros): {(best_pred[y_val == 0] == 0).mean()*100:.1f}%')
print(f'   • % subestimação: {(errors > 0).mean()*100:.1f}%')
print(f'   • % superestimação: {(errors < 0).mean()*100:.1f}%')

## 8. Feature Importance Analysis

In [ ]:
# Analisar importância das features do melhor modelo
if best_model_name in ['LightGBM', 'XGBoost', 'Random Forest'] and best_model is not None:
    
    if best_model_name == 'LightGBM':
        importance = best_model.feature_importance(importance_type='gain')
        feature_names = X_train.columns
    elif best_model_name == 'XGBoost':
        importance = best_model.feature_importances_
        feature_names = X_train.columns
    elif best_model_name == 'Random Forest':
        importance = best_model.feature_importances_
        feature_names = X_train.columns
    
    # Criar DataFrame com importâncias
    feature_imp_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importance
    }).sort_values('importance', ascending=False)
    
    # Visualizar top 20 features
    plt.figure(figsize=(12, 8))
    top_features = feature_imp_df.head(20)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Importância')
    plt.title(f'Top 20 Features Mais Importantes - {best_model_name}')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print(f'🔝 Top 15 Features Mais Importantes - {best_model_name}:')
    for i, (_, row) in enumerate(feature_imp_df.head(15).iterrows(), 1):
        print(f'   {i:2d}. {row["feature"]}: {row["importance"]:.4f}')
    
    # Categorizar features por tipo
    feature_categories = {
        'Lag': [f for f in feature_imp_df['feature'] if 'lag' in f],
        'Rolling': [f for f in feature_imp_df['feature'] if any(x in f for x in ['media', 'std', 'max', 'min'])],
        'Temporal': [f for f in feature_imp_df['feature'] if any(x in f for x in ['mes', 'semana', 'ano'])],
        'Histórico': [f for f in feature_imp_df['feature'] if any(x in f for x in ['historica', 'primeira', 'ultima', 'atividade'])],
        'Outros': [f for f in feature_imp_df['feature'] if f not in sum([v for v in [feature_categories.get(k, []) for k in ['Lag', 'Rolling', 'Temporal', 'Histórico']]], [])]
    }
    
    print(f'\n📋 Importância por Categoria de Features:')
    for category, features in feature_categories.items():
        if features:
            total_importance = feature_imp_df[feature_imp_df['feature'].isin(features)]['importance'].sum()
            print(f'   • {category}: {total_importance:.4f} ({len(features)} features)')
            
else:
    print('⚠️ Feature importance não disponível para este modelo')

## 9. Preparação para Predições Finais

In [ ]:
print('🎯 Preparação para predições finais...')

# Retreinar melhor modelo com todos os dados disponíveis
print(f'🔄 Retreinando {best_model_name} com todos os dados...')

# Preparar dados completos
X_full = dados[all_features].fillna(0)
y_full = dados[target]

if best_model_name == 'LightGBM':
    # Retreinar LightGBM
    train_full_lgb = lgb.Dataset(X_full, label=y_full)
    final_model = lgb.train(
        lgb_params,
        train_full_lgb,
        num_boost_round=lgb_model.best_iteration,
        verbose_eval=False
    )
    
elif best_model_name == 'XGBoost':
    # Retreinar XGBoost
    final_model = xgb.XGBRegressor(**xgb_params, n_estimators=xgb_model.best_iteration)
    final_model.fit(X_full, y_full, verbose=False)
    
elif best_model_name == 'Random Forest':
    # Retreinar Random Forest
    final_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        min_samples_split=20,
        min_samples_leaf=10,
        random_state=42,
        n_jobs=-1
    )
    final_model.fit(X_full, y_full)
    
else:
    # Usar estratégia baseline
    final_model = None
    combo_means_full = dados.groupby(['pdv_id', 'produto_id'])['quantidade'].mean().to_dict()
    global_mean_full = y_full.mean()

print('✅ Modelo final treinado e pronto para predições')

# Salvar modelo e configurações
model_artifacts = {
    'model': final_model,
    'model_type': best_model_name,
    'features': all_features,
    'target': target,
    'validation_mae': results_df.iloc[0]['MAE'],
    'validation_rmse': results_df.iloc[0]['RMSE'],
    'validation_r2': results_df.iloc[0]['R²'],
    'validation_wmape': results_df.iloc[0]['WMAPE'],
    'training_date': pd.Timestamp.now(),
    'combo_means': combo_means_full if best_model_name not in ['LightGBM', 'XGBoost', 'Random Forest'] else None,
    'metadata': metadata
}

# Salvar artefatos do modelo
with open('../data/trained_model.pkl', 'wb') as f:
    pickle.dump(model_artifacts, f)

print('💾 Modelo e artefatos salvos em: data/trained_model.pkl')
print('🎯 Pronto para gerar predições para o período de teste!')

## 10. Resumo e Próximos Passos

In [ ]:
print('🎉 MODELAGEM CONCLUÍDA COM SUCESSO!')
print('=' * 60)

print(f'\n🏆 Melhor Modelo: {best_model_name}')
print(f'   • MAE: {results_df.iloc[0]["MAE"]:.4f}')
print(f'   • RMSE: {results_df.iloc[0]["RMSE"]:.4f}')
print(f'   • R²: {results_df.iloc[0]["R²"]:.4f}')
print(f'   • WMAPE: {results_df.iloc[0]["WMAPE"]:.2f}%')

improvement_over_baseline = (results_df[results_df['Model'] == 'Média Simples']['MAE'].iloc[0] - results_df.iloc[0]['MAE']) / results_df[results_df['Model'] == 'Média Simples']['MAE'].iloc[0] * 100
print(f'   • Melhoria sobre baseline: {improvement_over_baseline:.1f}%')

print(f'\n📊 Comparação de Modelos:')
for i, (_, row) in enumerate(results_df.iterrows(), 1):
    print(f'   {i}. {row["Model"]}: MAE = {row["MAE"]:.4f}, WMAPE = {row["WMAPE"]:.2f}%')

print(f'\n💾 Artefatos Salvos:')
print('   ✅ trained_model.pkl - Modelo treinado e configurações')
print('   ✅ feature_engineering_metadata.pkl - Metadados do processamento')
print('   ✅ dados_features_completo.parquet - Dataset com features')

print(f'\n🔄 Próximos Passos:')
print('   1. 📅 Criar dados de teste para as 5 semanas de 2023')
print('   2. 🎯 Gerar predições usando o modelo treinado')
print('   3. 🆕 Aplicar estratégia para novas combinações (predição = 0)')
print('   4. 📋 Criar arquivo de submissão no formato requerido')
print('   5. 🧪 Validar predições e fazer análise final')

print(f'\n🚀 SISTEMA DE FORECASTING COMPLETO E PRONTO!')
print('   • Grid Inteligente com otimização de memória')
print('   • Features avançadas (30+ variáveis)')
print('   • Modelo ML de alta performance')
print('   • Validação temporal robusta')
print('   • Pipeline completo e automatizado')

print('\n✅ Modelagem e Treinamento CONCLUÍDOS!')